## Assignment 2 - Movie Classification, the sequel
![](https://images-na.ssl-images-amazon.com/images/S/sgp-catalog-images/region_US/paramount-01376-Full-Image_GalleryBackground-en-US-1484000188762._RI_SX940_.jpg)


#### In this assignment, we will learn a little more about word2vec and then use the resulting vectors to make some predictions.

We will be working with a movie synopsis dataset, found here: http://www.cs.cmu.edu/~ark/personas/

The overall goal should sound a little familiar - based on the movie synopses, we will classify movie genre. Some of your favorites should be in this dataset, and hopefully, based on the genre specific terminology of the movie synopses, we will be able to figure out which movies are which type.

### Task 1: clean your dataset!

For your input data:

1. Find the top 10 movie genres
2. Remove any synopses that don't fit into these genres
3. Take the top 10,000 reviews in terms of "Movie box office revenue"

Congrats, you've got a dataset! For each movie, some of them may have multiple classifications. To deal with this, you'll have to look at the Reuters dataset classification code that we used previously and possibly this example: https://github.com/keras-team/keras/blob/master/examples/reuters_mlp.py

We want to use categorical cross-entropy as our loss function (or a one vs. all classifier in the case of SVM) because our data will potentially have multiple classes!

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
import gensim
import word2vec

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import LSTM


Using TensorFlow backend.


In [2]:
dataPath = 'MovieSummaries/'
movieHeader = ['wikiID', 'freebaseID', 'name', 'releaseDate', 'revenue',
               'runtime', 'languages', 'countries', 'genres']
movieDat = pd.read_csv(dataPath + 'movie.metadata.tsv', delimiter = '\t',
                      header = None, names = movieHeader)
synopsisDat = pd.read_csv(dataPath + 'plot_summaries.txt', delimiter = '\t',
                      header = None, names = ['wikiID', 'synopsis'])

In [3]:
# To find top genres, will split the genres into their own columns
# since one move can be multiple genres and then sum those columns
# to find the max 10

### Step 1 -- Convert Genres into a list of genres
def cleanGenres(genreDat):
    clean = [re.findall(r'"\S+": "(.+)"', x) for x in genreDat.split(',')]
    return [item for sublist in clean for item in sublist]
    
movieDat['genres_clean'] = movieDat.genres.apply(cleanGenres)

### Step 2 --- "One Hot Encode" that list and then join back
mlb = MultiLabelBinarizer()
movieDat = movieDat.join(pd.DataFrame(mlb.fit_transform(movieDat['genres_clean']),
                          columns=mlb.classes_,
                          index=movieDat.index))

### Step 3 --- Find genres with the largest sums
idCols = movieHeader.copy()
idCols.extend(['genres_clean'])
genreCols = movieDat.columns.difference(idCols)
topGenres = movieDat.loc[:, genreCols].sum().nlargest(10).index

In [4]:
movieDat['topGenresOnly'] = movieDat['genres_clean'].apply(lambda x: set(x).intersection(topGenres))
containsGenreBool = movieDat['genres_clean'].apply(any)
movies = movieDat.loc[containsGenreBool].sort_values('revenue', ascending=False)
finalDat = movies.merge(synopsisDat, how = 'inner', on='wikiID').iloc[:10000]

# Split into X and y and preprocess
X = np.array(finalDat['synopsis'].apply(gensim.utils.simple_preprocess))
y = np.array(finalDat['topGenresOnly'])

all_words = set(w for words in X for w in words)
vocab_size = len(all_words)
embed_size = 200

### Task 2: Split the data

Make a dataset of 70% train and 30% test. Sweet.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

In [6]:
mlb2 = MultiLabelBinarizer()
train_labels = mlb2.fit_transform(y_train)
test_labels = mlb2.transform(y_test)

### Task 3a: Build a model using ONLY word2vec

Woah what? I don't think that's recommended...

In fact it's a commonly accepted practice. What you will want to do is average the word vectors that will be input for a given synopsis (https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html) and then input that averaged vector as your feature space into a model. For this example, use a Support Vector Machine classifier. For your first time doing this, train a model in Gensim and use the output vectors.


In [7]:
model = gensim.models.Word2Vec (X_train, size=100, window=5, min_count=5, workers=10)
model.train(X_train,total_examples=len(X_train),epochs=10)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [8]:
# Need to convert X_train and X_test to matrix of values by
# substituting each word to get its mean embedded score
def convert_word_mat_to_num(word_mat, w2v):
    dim = len(next(iter(w2v.values())))
    return np.array([np.mean([w2v[w] for w in words if w in w2v]
                             or [np.zeros(dim)], axis=0)
                     for words in word_mat])

In [9]:
def runMod(mod, embedding):
    clf = OneVsRestClassifier(mod(random_state=42))
    train_x = convert_word_mat_to_num(X_train, embedding)
    clf.fit(train_x, train_labels)
    
    test_x = convert_word_mat_to_num(X_test, embedding)
    preds = clf.predict(test_x)
    acc = metrics.accuracy_score(test_labels, preds)
    return (clf, acc)

In [10]:
svc_w2v = runMod(SVC, w2v)

### Task 3b: Do the same thing but with pretrained embeddings

Now pull down the Google News word embeddings and do the same thing. Compare the results. Why was one better than the other?

In [11]:
model_goog = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
w2v_goog = {w: vec for w, vec in zip(model_goog.wv.index2word, model_goog.wv.syn0)}

In [12]:
svc_w2v_goog = runMod(SVC, w2v_goog)

In [13]:
print('Accuracy of W2V: {}'.format(svc_w2v[1]))
print('Accuracy of Google: W2V: {}'.format(svc_w2v_goog[1]))

Accuracy of W2V: 0.17066666666666666
Accuracy of Google: W2V: 0.106


The word2vec model does better job classifying. This could be due to the fact that the pretrained is news focused and movie summaries are not inherently similar to a news article.

### Task 3: Build a neural net model using word2vec embeddings (both pretrained and within an Embedding layer from Keras)

In [14]:
def create_int_word_dict(model):
    mapdict = {}
    for i in range(len(model.wv.vocab)):
        word = model.wv.index2word[i]
        mapdict[word] = i
    return mapdict

In [15]:
def create_embed_matrix(model, all_words=all_words):
    "Create a weight matrix for words"
    vocab_size = len(all_words)
    embedding_matrix = np.zeros((vocab_size, model.vector_size))
    n = 0
    word_list = list(all_words)
    for i in range(vocab_size):
        word = word_list[i]
        if word in model.wv.vocab:
            embedding_vector = model.wv[word]
            if embedding_vector is not None:
                embedding_matrix[n] = embedding_vector
                n += 1

    return embedding_matrix[:n, :]

In [16]:
# Convert X datasets from words to numbers
def convert_word_to_num(word_mat, model, max_len = None):
    new_mat = []
    for review in word_mat:
        tmp = []
        for w in review:
            if w in model.wv.vocab:
                tmp.append(model.wv.vocab[w].index + 1)
            else:
                tmp.append(0)
        new_mat.append(tmp)

    return pad_sequences(new_mat, padding='post', maxlen=max_len)

In [22]:
def run_keras_model1(gensim_model, batch_size, 
                     create_emebed_mat = False):
    x_train = convert_word_to_num(X_train, gensim_model)
    x_test = convert_word_to_num(X_test, gensim_model, 
                                 max_len=x_train.shape[1])
    
    # For embedding layer
    if create_emebed_mat:
        embed_matrix = create_embed_matrix(gensim_model)
        vocab_size = embed_matrix.shape[0]
        e = Embedding(vocab_size, gensim_model.vector_size,
                      weights=[embed_matrix],
                     input_length=x_train.shape[1], trainable=False)
    else:
        e = gensim_model.wv.get_keras_embedding()
        e.input_length = x_train.shape[1]    
    
    # define model
    print('Build model...')
    model = Sequential()
    # e = Embedding(vocab_size, embed_size, weights=[embed_matrix], 
    #               input_length=maxlen, trainable=False)
    model.add(e)
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    # summarize the model
    print(model.summary())
    # fit the model
    model.fit(x_train, train_labels,
              batch_size=batch_size,
              epochs=5,
              validation_data=(x_test, test_labels))

    score, acc = model.evaluate(x_test, test_labels,
                                batch_size=batch_size)
    return (score, acc)

In [25]:
keras_w2v = run_keras_model1(model, 32)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3469, 100)         2608500   
_________________________________________________________________
flatten_6 (Flatten)          (None, 346900)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                3469010   
Total params: 6,077,510
Trainable params: 3,469,010
Non-trainable params: 2,608,500
_________________________________________________________________
None
Train on 7000 samples, validate on 3000 samples
Epoch 1/5
  32/7000 [..............................] - ETA: 1:07 - loss: 4.5563 - acc: 0.1562

InvalidArgumentError: indices[24,14] = 26085 is not in [0, 26085)
	 [[Node: embedding_6/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_6/embeddings/read, embedding_6/Cast)]]

Caused by op 'embedding_6/Gather', defined at:
  File "/Users/bradmckeen/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/bradmckeen/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/bradmckeen/anacon

In [26]:
keras_goog = run_keras_model1(model_goog, 32, create_emebed_mat=True)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3469, 300)         13400100  
_________________________________________________________________
flatten_7 (Flatten)          (None, 1040700)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                10407010  
Total params: 23,807,110
Trainable params: 10,407,010
Non-trainable params: 13,400,100
_________________________________________________________________
None
Train on 7000 samples, validate on 3000 samples
Epoch 1/5


InvalidArgumentError: indices[0,1] = 175828 is not in [0, 44667)
	 [[Node: embedding_7/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_7/embeddings/read, embedding_7/Cast)]]

Caused by op 'embedding_7/Gather', defined at:
  File "/Users/bradmckeen/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/bradmckeen/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/bradmckeen/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/bradmckeen/anacond

### Task 4: Change the architecture of your model and compare the result

### Task 5: For each model, do an error evaluation

You now have a bunch of classifiers. For each classifier, pick 2 good classifications and 2 bad classifications. Print the expected and predicted label, and also print the movie synopsis. From these results, can you spot some systematic errors from your models?